In [ ]:
# Implememtación base del algoritmo Dyna Q

import numpy as np
import random


In [ ]:
# parámetros
alpha = 0.1              # tasa de aprendizaje
gamma = 0.995             # factor de descuento
epsilon = 0.1            # epsilon
n_planning = 5           # planeación
num_episodios = 1500

In [ ]:
# tabla Q
q_shape = bins_por_variable + [2] 
Q = np.zeros(q_shape)

In [ ]:
for episodio in range(num_episodios):
    obs, _ = env.reset()
    estado_disc = discreto(obs)
    terminado = False
    recompensa_total = 0
    
    while not terminado:
        # epsilon-greedy
        if np.random.rand() < epsilon:
            accion = np.random.randint(2)
        else:
            accion = np.argmax(Q[estado_disc])
        
        
        obs_sig, recompensa, done, truncated, _ = env.step(accion)
        terminado = done or truncated
        recompensa_total = recompensa_total + recompensa
        pasos_totales = pasos_totales + 1
        
        estado_sig_disc = discreto(obs_sig)
        
        # actualizar Q 
        mejor_q_sig = np.max(Q[estado_sig_disc])
        td_error = recompensa + gamma * mejor_q_sig - Q[estado_disc + (accion,)]
        Q[estado_disc + (accion,)] = Q[estado_disc + (accion,)] + alpha * td_error
        Model[(estado_disc, accion)] = (recompensa, estado_sig_disc)
        
        # planeación
        for _ in range(n_planning):
            (s_plan, a_plan) = random.choice(list(Model.keys()))
            (r_plan, s_sig_plan) = Model[(s_plan, a_plan)]
            
            mejor_q_plan = np.max(Q[s_sig_plan])
            td_error_plan = r_plan + gamma * mejor_q_plan - Q[s_plan + (a_plan,)]
            Q[s_plan + (a_plan,)] = Q[s_plan + (a_plan,)] + alpha * td_error_plan
        
        estado_disc = estado_sig_disc
    
    
    lista_recompensas.append(recompensa_total)
    lista_pasos.append(pasos_totales)
    
    # (Opcional) Ajustar epsilon de forma decreciente
    epsilon = max(0.01, epsilon * 0.99)

    if (episodio + 1) % 25 == 0:
        print(f"Episodio {episodio+1}, Recompensa = {recompensa_total}")

env.close()

In [ ]:
def promedio(datos, vnv=10): # vnv: para el promedio de los episodios
    return np.convolve(datos, np.ones(vnv)/vnv, mode='valid')

vnv = 10
recompensas_soft = promedio(lista_recompensas, vnv)
pasos_soft = lista_pasos[vnv-1:]  